# Purpose: To build out the skeleton Pseudo Code for this project

__For now__: Moving my notes from my one one one with Elizabeth and will build out more in the next few days

1. Input .csv from experimental team to describe data
2. CSV pulls the proper images from NDP (or wherever best storage location is determined to be)
2a. There needs to be some integration with NDP to break the image into chunks with the appropriate zoom and then provide the original slice with locations of test and training images printed out.
2b. I think to do this we are going to have to register the image to some kind of atlas... or somethign to determine regions so that the code can grab pictures
3. Images need to be split
3a. Regionally
3b. Test Groups
3c. Into Small enough sizes for most efficient processing by package
3d. Magifications
4. Threshold and segment
4a. Ifthresholds (for general object identification) [July]
4b. + Skeletonization (for specific highly branched morphometric analysis) [Robin]
5. Split data into testing and training groups
6. Shape factor analysis on both models (before ML model)
7. Feed 4(1) and 4(2) separately to modified VAMPIRE package to build models
8. Store created models
9. Test models with testing data sets
10. Data output
11. Comprehensive CSV for optional additional analysis
12. Data Visualization
13. Save all data into some storage location (Google Drive?)


Other Notes: 
1. Modified VAMPIRE package: Worth forking the Wirtz lab repository now and building on by ourselves possibly – adding that fork to our repository - COMPLETE
2. Want a repository that works through Binder? Preferably so the Neonatalogy lab can do this straight through a web interface
3. Along with outputs want to output the variables used for all of the steps on days that experiments were run with an easy print out maybe for lab notebook storage? (A way to integrate the electronic lab notebook I want to get more fine tuned for our lab with a standard lab notebook - and provide some under the hood knowledge for Neonatology)

4. Add in an optimization step for what size of image is sufficient for cropping down the whole scan images 
5. Including an analysis before creating that analysis that selects the best # of shape modes and #coordinate points – maybe we could reach out the Denis Wirtz lab about this or build in our own integration



Current Goal:

1. Get a bunch of images from a slice of the ferret brain
2. Put them in a folder
3. Perform thresholding within the Jupyter Notebook
3. Integrate vampire into the Jupyter Notebook
4. Run those images and get an output within the notebook or specific folder (which should will need a results output)
5. Save all of this information to some results folder
6. See if it works in binder

Next Goals:
1. Build out ifthresholds more for immunohistochemistry stains
2. Build in NDP regional registration and automatic image breakdown
3. Integrate Google Drive
4. New visualizations based on what came from paper
5. Statistics possibly with Rthon
6. Speed up any slow processes with Cython
7. Integrate ifThresholds to pick and perform the best thresholding (or integrate a step to say whether this needs to be done or not)

*Step 1: Import necessary packages*

In [42]:
import shutil, os
import re

import numpy as np
import pandas as pd

from skimage import io
from skimage import filters
from skimage import measure
from skimage import morphology
from skimage import segmentation
from skimage import color

%matplotlib inline
import matplotlib.pyplot as plt
#plt.style.use('seaborn-white')

from sklearn.model_selection import train_test_split

from PIL import Image
import image_slicer

*Step 2: User Inputs*

In [93]:
#Data folder to the stain you are running morphology on (right now only works one stain at a time)
#rerun for each individual stain

#Folder breakdown: stain>animal>region>actual images
data_folder = '/Users/hhelmbre/Desktop/KH_LPS_ferret'

#Image type of your images (they should all be the same type)
image_file_type = '.tif'
file_type_new = '.png'

#Enter the name of your stain as a string
stain1 = 'Iba1' #c1
stain2 = 'dapi' #c2

experiment_name = 'practice'

region_list = ['corpus_callosum', 'hippocampus', 'cortex', 'white_matter', 'basal_ganglia', 'thalamus']
treatment_list = ['NT', 'AcAc', 'OGD2h', 'Epo', 'OGD1h']
female_list = ['68-24', '68-28', '68-35', '68-36', '68-53', '68-60', '44-48', '46-45', '46-47', '64-19', '68-37', '68-43', '68-49', '68-20']
male_list = ['68-6_', '68-6b', '68-11', '68-12', '68-18', '68-68', '44-42', '46-39', '46-41', '60-1', '64-1', '64-43']

*Step X: Threshold and segment everything and then split test and train*

In [44]:
folder_list = os.listdir(data_folder)
folder_list = np.asarray(folder_list)

In [45]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for folders in folder_list:
    if mac_annoyance in str(folders):
        folder_list = np.delete(folder_list, (k), axis=0)
    else:
        k+=1

*Step 1: Split up the stains and put them into their own folders*

*Step 1b: Split the images into their individual stains*

In [46]:
for non_channel_split_images in folder_list:
    name = str(data_folder + '/' + non_channel_split_images)
    im = io.imread(name)
    channel1 = im[0, :, :]
    channel2= im[1, :, :]
    filename = non_channel_split_images.replace(image_file_type, "")
    channel1 = Image.fromarray(np.uint16(channel1))
    channel1.save(str(data_folder + '/' + filename + '_' + stain1 + file_type_new))
    channel2 = Image.fromarray(np.uint16(channel2))
    channel2.save(str(data_folder + '/' + filename + '_' + stain2 + file_type_new))
    

*Step 1c: Split the images into 4ths*

In [47]:
arr = os.listdir(data_folder)
file_list = np.asarray(arr)

In [48]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list:
    if mac_annoyance in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

In [49]:
k=0
for images in file_list:
    if '.tif' in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

In [50]:
slice_number = 4
for files in file_list:
    image_slicer.slice(str(data_folder + '/' + files), slice_number)

In [51]:
arr = os.listdir(data_folder)
file_list = np.asarray(arr)

In [52]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list:
    if mac_annoyance in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

*Move split images into their proper stain folders*

In [53]:
os.mkdir(str(data_folder + '/' + stain1))
os.mkdir(str(data_folder + '/' + stain2))

In [54]:
for tiled_images in file_list:
    conditional = str(str(tiled_images)[-6].isdigit())
    if conditional == 'True':
        if stain1 in tiled_images:
            shutil.move(str(data_folder + '/' + tiled_images), str(data_folder + '/' + stain1 + '/' + tiled_images))
        elif stain2 in tiled_images:
            shutil.move(str(data_folder + '/' + tiled_images), str(data_folder + '/' + stain2 + '/' + tiled_images))
    else:
        pass

*Split into the proper sexes*

In [65]:
arr = os.listdir(str(data_folder + '/' + stain1))
file_list_stain1 = np.asarray(arr)

In [66]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list_stain1:
    if mac_annoyance in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

In [67]:
os.mkdir(str(data_folder + '/' + stain1 + '/' + 'female'))
os.mkdir(str(data_folder + '/' + stain1 + '/' + 'male'))

In [68]:
for nonsexed_images in file_list_stain1:
    new_data_folder = data_folder + '/' + stain1 + '/'
    for animal_codes in female_list:
        if animal_codes in nonsexed_images:
            shutil.move(str(new_data_folder + nonsexed_images), str(new_data_folder + 'female/' + nonsexed_images))
        else:
            pass


In [94]:
arr = os.listdir(str(data_folder + '/' + stain1))
file_list_stain1 = np.asarray(arr)

In [95]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list_stain1:
    if mac_annoyance in str(images):
        file_list_stain1 = np.delete(file_list_stain1, (k), axis=0)
    else:
        k+=1

In [96]:
k = 0
for images in file_list_stain1:
    if '.png' in images:
        k+=1
    else:
        file_list_stain1 = np.delete(file_list_stain1, (k), axis=0)

In [97]:
for nonsexed_images in file_list_stain1:
    new_data_folder = data_folder + '/' + stain1 + '/'
    for animal_codes in male_list:
        if animal_codes in nonsexed_images:
            shutil.move(str(new_data_folder + nonsexed_images), str(new_data_folder + 'male/' + nonsexed_images))            
        else:
            pass

*Split into treatment groups*

In [62]:
arr = os.listdir(str(data_folder + '/' + stain1))
file_list_stain1 = np.asarray(arr)

In [ ]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list_stain1:
    if mac_annoyance in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

In [ ]:
for treatments in treatment_list:
    os.mkdir(str(data_folder + '/' + stain1 + '/' + regions))

*Split Regions within each folder*

In [14]:
arr = os.listdir(str(data_folder + '/' + stain1))
file_list_stain1 = np.asarray(arr)

In [15]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list_stain1:
    if mac_annoyance in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

In [26]:
for regions in region_list:
    os.mkdir(str(data_folder + '/' + stain1 + '/' + regions))

In [36]:
for stain1_images in file_list_stain1:
    region_code = str(stain1_images[12])
    region_folder = str(data_folder + '/' + stain1 + '/')
    if region_code == '1':
        shutil.move(str(region_folder + stain1_images), str(region_folder + region_list[0] + '/' + stain1_images))
    elif region_code == '2':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[1] + '/' + stain1_images))
    elif region_code == '3':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[2] + '/' + stain1_images))
    elif region_code == '5':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[3] + '/' + stain1_images))
    elif region_code == '6':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[4] + '/' + stain1_images))
    elif region_code == '7':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[5] + '/' + stain1_images))
    else:
        pass
        

In [37]:
for stain1_images in file_list_stain1:
    region_code = str(stain1_images[11])
    region_folder = str(data_folder + '/' + stain1 + '/')
    if region_code == '1':
        shutil.move(str(region_folder + stain1_images), str(region_folder + region_list[0] + '/' + stain1_images))
    elif region_code == '2':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[1] + '/' + stain1_images))
    elif region_code == '3':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[2] + '/' + stain1_images))
    elif region_code == '5':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[3] + '/' + stain1_images))
    elif region_code == '6':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[4] + '/' + stain1_images))
    elif region_code == '7':
        shutil.move(str(region_folder + stain1_images),str(region_folder + region_list[5] + '/' + stain1_images))
    else:
        pass

In [5]:
#Folder breadown should be (1) stains, (2) animals, (3) regions
#this code gets a list of the individual paths for each region folder with images
threshold_paths = []
for animals in folder_list:
    region_list = os.listdir(str(data_folder + '/' + animals))
    region_list = np.asarray(region_list)
    
    k=0
    for regions in region_list:
        if mac_annoyance in str(regions):
            region_list = np.delete(region_list, (k), axis=0)
        else:
            k+=1
    
    for regions in region_list:
        path = str(data_folder + '/' + animals + '/' + regions)
        threshold_paths.append(path)

*Step 3: Threshold and Segment Images*

Step 3a: Import and threshold images from 'data_folder' from downloaded NDP images

*need to go in and add code to create a reasonable cut off for the min_size remove small objects*

*Go in and add code that does a better job at separating overlapping cells (maybe this is just in thresholding*)

In [6]:
for paths in threshold_paths:

    #Creating a folder for the new tresholded images
    os.mkdir(str(paths + '/' + 'threshold_applied_images'))

    #initializing a pandas dataframe
    overall_region_meas = pd.DataFrame(columns = ['Filename', 'ImageID', 'ObjectID', 'X', 'Y', 'Area', 'Perimeter', 'Major Axis', 'Minor Axis', 'Circularity', 'Aspect Ratio'])

    #initializing a count
    
    image_list = os.listdir(str(paths))
    image_list = np.asarray(image_list)
    
    k=0
    for images in image_list:
        if image_file_type in str(images):
            k+=1
        else:
            image_list = np.delete(image_list, (k), axis=0)
    
    
    #Initializing an imag count
    k=1
    
    #Looping through all images in list (this task should be put to multiple CPUs in future if a large data set)
    for images in image_list:

        #initializing a pandas data frame for measurement data
        region_meas = pd.DataFrame()

        #Going through each of the images to get their binarized images and measurement info
        name = str(paths + '/' + images)
        im = io.imread(name)
        im = im[:,:,0]
        threshold = filters.threshold_otsu(im)
        binary = morphology.closing(im < threshold, morphology.square(5))
        label_image = measure.label(binary, return_num=False, connectivity=2)
        binary2 = morphology.remove_small_objects(label_image, min_size=400, connectivity=2, in_place=True)
        invert_binary2 = np.invert(binary2)

        # Saving the thresheld images to their own folder with modified names
        im_to_save = Image.fromarray(np.uint8(binary2), mode='L')
        new_name = str(paths + '/' + 'threshold_applied_images' + '/' + images)
        new_name = new_name.replace('.jpg','_threshold_ch1.jpg')
        im_to_save.save(new_name)


        #Saving the region properties as a csv
        props = measure.regionprops_table(binary2, properties= ('centroid', 'area', 'perimeter', 'major_axis_length', 'minor_axis_length'))
        props_meas = pd.DataFrame(props)
        region_meas['X'] = props_meas['centroid-0']
        region_meas['Y'] = props_meas['centroid-1']
        region_meas['Area'] = props_meas['area']
        region_meas['Perimeter'] = props_meas['perimeter']
        region_meas['Major Axis'] = props_meas['major_axis_length']
        region_meas['Minor Axis'] = props_meas['minor_axis_length']
        region_meas['Circularity'] = (region_meas['Area']*4*np.pi)/(region_meas['Perimeter']**2)
        region_meas['Aspect Ratio'] = (region_meas['Major Axis']/region_meas['Minor Axis'])
        region_meas.insert(0, 'Filename', images)
        region_meas.insert(1, 'ImageID', k)
        region_meas.insert(2, 'ObjectID', np.arange(len(region_meas)))

        overall_region_meas = overall_region_meas.append(region_meas)
        k+=1

    path = str(paths + '/' + 'threshold_applied_images/')
    overall_region_meas.to_csv(path + 'c1_registry.csv', index=False)
                           

*Step X: Test and Train Split before Thresholding*

In [7]:
#Go through all the threshold paths + the threshold folder and assign 8 of them for training and 2 of them for testing

In [8]:
test_paths = []
for animals in folder_list:
    region_list = os.listdir(str(data_folder + '/' + animals))
    region_list = np.asarray(region_list)
    
    k=0
    for regions in region_list:
        if mac_annoyance in str(regions):
            region_list = np.delete(region_list, (k), axis=0)
        else:
            k+=1
    
    for regions in region_list:
        path = str(data_folder + '/' + animals + '/' + regions + '/' + 'threshold_applied_images/')
        test_paths.append(path)


In [9]:
os.mkdir(str(data_folder + '/' + 'train'))

name_count = 0

for paths in test_paths:
    
    image_list = os.listdir(str(paths))
    image_list = np.asarray(image_list)
    
    k=0
    for images in image_list:
        if image_file_type in str(images):
            k+=1
        else:
            image_list = np.delete(image_list, (k), axis=0)

    X_train, X_test= train_test_split(image_list, test_size=0.20, random_state=1)
    
    test_name_count = 1

    for names in image_list:
        
        if names in X_train[:]:
            shutil.move(str(paths + '/' + names), str(data_folder + '/' + 'train'))
            
            
            train_name = str(data_folder + '/' + 'train/' + names)
            string_count = str(name_count)
            replace_name = str('xy' + string_count + stain + 'ch1' + '.jpg')
            os.rename(str(data_folder + '/' + 'train/' + names), str(data_folder + '/' + 'train/' + replace_name))
            
            
            name_count +=1
            
            
        else:
            #renaming the testing groups
            string_test_name_count = str(test_name_count)
            os.rename(str(paths + '/' + names), str(paths + '/' + 'xy' + string_test_name_count + 'ch1' + '.jpg'))
            
            test_name_count += 1
            

# Step 3: Vampire Analysis

*The following will pull up the GUI as a popup - I want to be able to input values into the GUI straight from here not have to point and click*

#We are going to work with what we have an have a print out here of what to input in the VAMPIRE GUI - it is still faster than it used to be. 

(1) Build and then apply model with the vampire GUI (will work on making it not GUI later)

*Step X: Create the CSVs for Building and applying the model*

In [10]:
#create a model repository to store the output information
#create a csv for building the model
#create a csv for applying the model

In [11]:
#creates the directory in your data folder to put all information related to the model
os.mkdir(str(data_folder + '/' + 'model'))

In [12]:
#creates the csv for building a model
data = [['1', stain, str(data_folder + '/' + 'train'), 'ch1', 'iba1']]
build_model_csv = pd.DataFrame(data, columns = ['set ID', 'condition', 'set location', 'tag', 'note']) 

#saves csv to newly created model directory
build_model_csv.to_csv(data_folder + '/' + 'model/' + 'images_to_build_model.csv', index=False)

In [13]:
#creates the csv for applying a model
apply_model_csv = pd.DataFrame(columns = ['set ID', 'condition', 'set location', 'tag', 'note'])

set_number = 1
for folders in test_paths:
    str_set_number = str(set_number)
    pattern = str(data_folder + '/' + '(.*?)' + '/threshold_applied_images/')
    substring = re.search(pattern, folders).group(1)

    df2 = pd.DataFrame({'set ID': [set_number], 'condition': [stain], 'set location': [folders], 'tag': ['ch1'], 'note': ['iba1']})
    apply_model_csv = apply_model_csv.append(df2)
    
    set_number += 1
    
apply_model_csv.to_csv(data_folder + '/' + 'model/' + 'images_to_apply_model.csv', index=False)

Step X: Print the inputs that should be added to VAMPIRE

In [17]:
print('Build Model CSV Path:', str(data_folder + '/' + 'model/' + 'images_to_build_model.csv'))

print('Number of Shape Models (Recommended):', '5')
print('Number of Shape Coordinates (Recommended):', '50')
print('Model Name:', experiment_name)

print('Apply Model CSV Path:', str(data_folder + '/' + 'model/' + 'images_to_apply_model.csv'))
print('Model to Apply:', str(data_folder + '/' + 'model/' + experiment_name))

Build Model CSV Path: /Users/hhelmbre/Desktop/6-17-2020-fiberfimages/iba1/model/images_to_build_model.csv
Number of Shape Models (Recommended): 5
Number of Shape Coordinates (Recommended): 50
Model Name: practice
Model to Apply: /Users/hhelmbre/Desktop/6-17-2020-fiberfimages/iba1/model/practice


*Step X: Open the VAMPIRE GUI, build, and then apply the model*

In [3]:
import vampireanalysis
from vampireanalysis import vampire

In [4]:
vampire()

## getboundary.py


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/hhelmbre/opt/anaconda3/envs/vampirenv/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'ch1'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/hhelmbre/opt/anaconda3/envs/vampirenv/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/Users/hhelmbre/opt/anaconda3/envs/vampirenv/lib/python3.7/site-packages/vampireanalysis/vampire.p

*Step 8: New visualization of VAMPIRE data*

- This should probably be built into VAMPIRE and not just into this notebook - think about this for the future

There still needs to be visualization steps here because what is output by vampire just is not great